# Computing relative dates in pandas

In [11]:
import pandas as pd
import numpy as np
from pandas._testing import makeTimeDataFrame

In [57]:
# Dummy dataframe with date as index
count = 10000
f_count = 10
frames = []
for _ in range(f_count):
    frames.append(makeTimeDataFrame(count))
    
df = pd.concat(frames, sort = False)

# Add a random ID-column
df['ID'] = np.random.randint(1, 4, df.shape[0])
# Drop unnecessary columns
df = df.drop(['A', 'B', 'C', 'D'], axis = 1)
df.head()

,ID
2000-01-03,1
2000-01-04,2
2000-01-05,2
2000-01-06,2
2000-01-07,3


In [58]:
# Drop 30% of the rows
df = df.sample(frac = .7)
len(df)

70000

In [59]:
# put the date in it's own column
df['date'] = df.index

# reset the index
df = df.reset_index(drop = True)
df.head()

,ID,date
0,2,2001-08-08
1,1,2032-06-17
2,2,2001-08-02
3,2,2004-11-29
4,3,2022-01-26


In [60]:
df = df.sort_values(by = ['date'])

In [61]:
lookup = {}

def do_date_differences(row, col, date_col = 'date'):
    if row[col] in lookup:
        first = lookup[row[col]]
    else:
        first = df.loc[df[col].isin([row[col]])].iloc[0][date_col]
        lookup[row[col]] = first
    second = row[date_col]
    diff = second - first
    return diff


# axis = 1, for rows
df['difference'] = df.apply(lambda row: do_date_differences(row, 'ID', 'date'), axis=1)
df

,ID,date,difference
51869,2,2000-01-03,0 days
3532,1,2000-01-03,0 days
51257,1,2000-01-03,0 days
61486,3,2000-01-03,0 days
24548,3,2000-01-03,0 days
...,...,...,...
24049,2,2038-04-30,13997 days
47491,2,2038-04-30,13997 days
53578,1,2038-04-30,13997 days
60701,2,2038-04-30,13997 days


In [62]:
#lookup

In [63]:
df[df['ID'] == 1 ]

,ID,date,difference
3532,1,2000-01-03,0 days
51257,1,2000-01-03,0 days
104,1,2000-01-03,0 days
68130,1,2000-01-04,1 days
7255,1,2000-01-04,1 days
...,...,...,...
28199,1,2038-04-29,13996 days
57530,1,2038-04-30,13997 days
22120,1,2038-04-30,13997 days
33104,1,2038-04-30,13997 days


In [64]:
df[df['difference'] > '15 days']

,ID,date,difference
10236,2,2000-01-19,16 days
4779,2,2000-01-19,16 days
31512,1,2000-01-19,16 days
68589,3,2000-01-19,16 days
50030,3,2000-01-19,16 days
...,...,...,...
24049,2,2038-04-30,13997 days
47491,2,2038-04-30,13997 days
53578,1,2038-04-30,13997 days
60701,2,2038-04-30,13997 days


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70000 entries, 51869 to 40003
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   ID          70000 non-null  int64          
 1   date        70000 non-null  datetime64[ns] 
 2   difference  70000 non-null  timedelta64[ns]
dtypes: datetime64[ns](1), int64(1), timedelta64[ns](1)
memory usage: 2.1 MB
